In [207]:
import torch
import pandas as pd
import re
import tiktoken
import importlib
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn

In [208]:
raw_data = open(r'./sample_data/harry_potter.txt', 'rt').read()


In [209]:
print(len(raw_data))
print(raw_data[:100])

20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [210]:

token = re.split(r'([,.:;?_!"()\']|--|\s)', raw_data)
unique_tokens = sorted(set(token))
vocab = {}
for integer, token in enumerate(token):
    vocab[token] = integer
print(vocab)

{'I': 9170, ' ': 9229, 'HAD': 2, 'always': 7520, 'thought': 9032, 'Jack': 4746, 'Gisburn': 5756, 'rather': 8656, 'a': 9148, 'cheap': 16, 'genius': 18, '--': 9213, 'though': 312, 'good': 606, 'fellow': 4902, 'enough': 9112, 'so': 8842, 'it': 9190, 'was': 8814, 'no': 9220, 'great': 8384, 'surprise': 5388, 'to': 9114, 'me': 9194, 'hear': 8484, 'that': 9168, ',': 9205, '': 9234, 'in': 9048, 'the': 9158, 'height': 144, 'of': 9228, 'his': 9136, 'glory': 6778, 'he': 9074, 'had': 8008, 'dropped': 5820, 'painting': 9180, 'married': 1034, 'rich': 1572, 'widow': 86, 'and': 9208, 'established': 92, 'himself': 8998, 'villa': 100, 'on': 9140, 'Riviera': 1200, '.': 9231, '(': 1533, 'Though': 112, 'would': 9000, 'have': 9002, 'been': 9020, 'Rome': 128, 'or': 7526, 'Florence': 132, ')': 1541, '\n': 9125, '"': 9233, 'The': 9198, 'what': 9028, 'women': 1952, 'called': 8224, 'can': 8180, 'Mrs': 8806, 'Gideon': 180, 'Thwing': 728, 'last': 7622, 'Chicago': 188, 'sitter': 190, 'deploring': 192, 'unaccountabl

In [211]:
class Token_maker:
  def __init__(self,data):
    self.string_to_int = data
    self.int_to_string = {}
    for s, i in data.items():
        self.int_to_string[i] = s
        #encoding the input to tokens
  def string_token(self, text):
      tokens = re.split(r'([,.:;?_!"()\']|--|\s)', text)
      token_ids = []
      for t in tokens:
            if t in self.string_to_int:
                token_ids.append(self.string_to_int[t])
      return token_ids
      #token to text
  def token_string(self, ids):
      words = []  #list to store reconstructed words
      for i in ids:
            if i in self.int_to_string:
                words.append(self.int_to_string[i])

      text = " ".join(words)  # join words with spaces
      text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
      return text


In [212]:
token_tests = Token_maker(token_id)
token_tests.string_token("For a famous place, it was very dark and shabby. A few old women were sitting in a corner, drinking tiny glasses of sherry. One of them was smoking a long pipe. A little man in a top hat was talking to the old bartender, who was quite bald and looked like a toothless walnut. The")



[38,
 2,
 118,
 2,
 2,
 776,
 8,
 0,
 2,
 588,
 2,
 1080,
 2,
 1068,
 2,
 316,
 2,
 160,
 2,
 10,
 0,
 2,
 14,
 2,
 440,
 2,
 729,
 2,
 1115,
 2,
 1091,
 2,
 2,
 571,
 2,
 118,
 2,
 293,
 8,
 0,
 2,
 2,
 2,
 2,
 725,
 2,
 10,
 0,
 2,
 2,
 725,
 2,
 993,
 2,
 1080,
 2,
 2,
 118,
 2,
 642,
 2,
 10,
 0,
 2,
 14,
 2,
 638,
 2,
 659,
 2,
 571,
 2,
 118,
 2,
 2,
 2,
 1080,
 2,
 977,
 2,
 1019,
 2,
 991,
 2,
 729,
 2,
 8,
 0,
 2,
 1100,
 2,
 1080,
 2,
 816,
 2,
 2,
 160,
 2,
 646,
 2,
 631,
 2,
 118,
 2,
 2,
 10,
 0,
 2,
 96]

In [213]:
token_tests.token_string([56, 2, 533, 2, 670, 2, 990, 2, 70, 10, 0, 2, 41, 2, 1080])

'I   have   mentioned   that   Mrs.    Gisburn   was'

In [214]:
print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.8.0


In [215]:
tokenizer = tiktoken.get_encoding("gpt2")

In [216]:
tokenizer.encode("kjewlkfjwlkejflkwjelkfjweklfj")

[42421,
 413,
 75,
 74,
 69,
 73,
 40989,
 365,
 73,
 2704,
 46265,
 73,
 417,
 74,
 69,
 73,
 732,
 74,
 1652,
 73]

In [217]:
batch = []
text_1 = "hello world, harry potter is here"
text_2 = "the harry potter, books are good"
batch.append(torch.tensor(tokenizer.encode(text_1)))
batch.append(torch.tensor(tokenizer.encode(text_2)))
batch = torch.stack(batch, dim=0)
print(batch)


tensor([[31373,   995,    11,  3971,   563,  1787,   353,   318,   994],
        [ 1169,  3971,   563,  1787,   353,    11,  3835,   389,   922]])


In [218]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [224]:


GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}



In [225]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # A simple placeholder

    def forward(self, x):
        # This block does nothing and just returns its input.
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # The parameters here are just to mimic the LayerNorm interface.

    def forward(self, x):
        # This layer does nothing and just returns its input.
        return x



In [226]:
torch.manual_seed(123)
dummy_model = DummyGPTModel(GPT_CONFIG_124M)
logits = dummy_model(batch)
print(logits)

tensor([[[-0.4730,  0.9534, -0.1357,  ..., -0.7575, -0.3694, -0.0449],
         [-0.1691,  1.1632,  0.3353,  ..., -0.2662,  1.3028,  0.6348],
         [ 0.9966,  1.0554,  0.4811,  ...,  1.2790,  0.3781, -0.3264],
         ...,
         [ 0.6231, -0.1986, -0.6411,  ...,  0.8808,  1.2568,  0.0774],
         [ 0.3620,  0.0164, -0.5234,  ..., -0.5839, -1.2104, -0.6022],
         [-0.2765, -0.2690, -1.3648,  ..., -1.3578, -1.2946, -0.1746]],

        [[-0.0740,  0.3409, -0.2020,  ..., -0.9521, -0.0818, -0.9191],
         [ 0.0391,  0.8940, -0.0082,  ...,  0.6117, -0.1145,  1.4780],
         [ 0.1847,  1.3747,  0.4571,  ...,  0.1258,  1.1706, -0.1840],
         ...,
         [ 0.4712,  0.3927, -0.1322,  ...,  0.5540,  0.2059,  0.5349],
         [ 0.0755, -0.3041,  0.7430,  ..., -0.5034, -0.3173, -0.9137],
         [ 0.4292, -0.4279, -0.8059,  ...,  0.7091, -1.0550, -0.5708]]],
       grad_fn=<UnsafeViewBackward0>)
